In [13]:
from urllib.request import urlopen
import os
from dotenv import load_dotenv
import json
import pandas as pd
import requests
from IPython.display import display

In [5]:
if __name__ == "__main__":
    load_dotenv()
    api_key = os.environ.get("api_key")
    params = {
        "frequency": "monthly",
        "start": "2024-01",
        "end":"2024-02",
        "sort[0][column]": "period",
        "offset": "0",
        "length": "5000",
        "api_key": api_key
    } 

ANf55hWD1ayUpkkQolvUADtq2sMhU83p2VZZlTql


In [17]:
# Set the URL
                   
url = f"https://api.eia.gov/v2/petroleum/pri/spt/data/?data[0]=value&sort[0][direction]=desc"

response = requests.get(
    url=url, params=params
)

response_data = response.json()

df_petrolium = pd.json_normalize(data=response_data)
df_petrolium =pd.DataFrame(df_petrolium["response.data"][0])
pd.options.display.max_columns = None
display(df_petrolium.head(5))



,period,duoarea,area-name,product,product-name,process,process-name,series,series-description,value,units
0,2024-08,Y35NY,NEW YORK CITY,EPD2F,No 2 Fuel Oil / Heating Oil,PF4,Spot Price FOB,EER_EPD2F_PF4_Y35NY_DPG,New York Harbor No. 2 Heating Oil Spot Price F...,2.163,$/GAL
1,2024-08,Y35NY,NEW YORK CITY,EPMRU,Conventional Regular Gasoline,PF4,Spot Price FOB,EER_EPMRU_PF4_Y35NY_DPG,New York Harbor Conventional Gasoline Regular ...,2.329,$/GAL
2,2024-08,YCUOK,NA,EPCWTI,WTI Crude Oil,PF4,Spot Price FOB,RWTC,"Cushing, OK WTI Spot Price FOB (Dollars per Ba...",76.68,$/BBL
3,2024-08,RGC,NA,EPMRU,Conventional Regular Gasoline,PF4,Spot Price FOB,EER_EPMRU_PF4_RGC_DPG,U.S. Gulf Coast Conventional Gasoline Regular ...,2.301,$/GAL
4,2024-08,Y35NY,NEW YORK CITY,EPD2DXL0,No 2 Diesel Low Sulfur (0-15 ppm),PF4,Spot Price FOB,EER_EPD2DXL0_PF4_Y35NY_DPG,New York Harbor Ultra-Low Sulfur No 2 Diesel S...,2.28,$/GAL


In [ ]:
##load data
postgres_clinet = PostgresQL